# TechStream Solutions

## 0. Import data

In [2]:
import pandas as pd

In [3]:
google_sheet_id = '10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw'
url = 'https://docs.google.com/spreadsheets/d/' + google_sheet_id + '/export?format=xlsx'
monthly_expense = pd.read_excel(url, sheet_name='Sheet1')

In [4]:
google_sheet_id = '1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4'
url = 'https://docs.google.com/spreadsheets/d/' + google_sheet_id + '/export?format=xlsx'
payroll = pd.read_excel(url, sheet_name='Sheet1')

In [5]:
google_sheet_id = '1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c'
url = 'https://docs.google.com/spreadsheets/d/' + google_sheet_id + '/export?format=xlsx'
daily_marketing = pd.read_excel(url, sheet_name='Sheet1')

In [6]:
google_sheet_id = '1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE'
url = 'https://docs.google.com/spreadsheets/d/' + google_sheet_id + '/export?format=xlsx'
payments = pd.read_excel(url, sheet_name='Sheet1')

In [7]:
google_sheet_id = '1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI'
url = 'https://docs.google.com/spreadsheets/d/' + google_sheet_id + '/export?format=xlsx'
lifespan = pd.read_excel(url, sheet_name='Sheet1')

## 1. CAC: Customer Acquisition Cost

In [8]:
monthly_expense.sample(5)

,#,month,category,item,amount
18,19,2023-03-01,Server Costs,AWS Hosting,8400
16,17,2023-02-01,Other,Office Supplies,550
15,16,2023-02-01,Office Rental,Office Rent,10000
7,8,2023-01-01,Other,Office Supplies,500
24,25,2023-03-01,Office Rental,Office Rent,10000


In [9]:
expense_202303 = monthly_expense[monthly_expense['month'] == '2023-03-01']
#expense_202303

### 1.1 Marketing Software Expense

In [10]:
crm_expense = expense_202303[expense_202303['item'] == 'Salesforce']['amount'].sum()
crm_expense

np.int64(1700)

### 1.2 Sales & Marketing Salaries

In [11]:
payroll.sample(5)

,month,department,employee_name,position,paid
4,2023-01-01,Marketing,Alice Johnson,Marketing Manager,1650
24,2023-02-01,Engineering,Diana Lee,Developer,1700
3,2023-01-01,Sales,Laura Miller,Sales Associate,800
50,2023-03-01,Analytics,David Lee,Data Analyst,1300
33,2023-02-01,Analytics,David Lee,Data Analyst,1300


In [12]:
payroll_202303 = payroll[payroll['month'] == '2023-03-01']

In [13]:
#sale_marketing_salary = payroll_202303[payroll_202303['department'].isin(['Sales', 'Marketing'])]['paid'].sum()
sale_marketing_salary = payroll_202303[(payroll_202303['department'] == 'Sales') |\
                                       (payroll_202303['department'] == 'Marketing') ]['paid'].sum()
sale_marketing_salary

np.int64(5950)

### 1.3 Daily Marketing

In [14]:
daily_marketing.sample(5)

,date,channel,spending
28,2023-01-08,Google Ads,945
14,2023-01-04,LinkedIn Ads,170
30,2023-01-08,LinkedIn Ads,877
127,2023-02-01,Twitter Ads,483
250,2023-03-04,LinkedIn Ads,801


In [15]:
daily_marketing_202303 = daily_marketing[(daily_marketing['date'] >= '2023-03-01') & (daily_marketing['date'] <= '2023-03-31')]

In [16]:
daily_marketing_expense = daily_marketing_202303['spending'].sum()
daily_marketing_expense

np.int64(68830)

### 1.4 Number of New Customers

In [17]:
payments.sample(5)

,date,customer_id,receipt_amount,new_customer
69,2023-01-08,2718,158,1
617,2023-02-28,1197,322,0
865,2023-03-25,1000,489,0
240,2023-01-24,1707,158,0
82,2023-01-10,2973,402,1


In [18]:
payments_202303 = payments[(payments['date'] >= '2023-03-01') & (payments['date'] <= '2023-03-31')]

In [19]:
number_of_new_customers = payments_202303[payments_202303['new_customer'] == 1]['customer_id'].nunique()
number_of_new_customers

63

### 1.5 CAC Calculation

In [20]:
total_acquistion_cost = crm_expense + sale_marketing_salary + daily_marketing_expense
cac = round(total_acquistion_cost / number_of_new_customers, 2)
cac

np.float64(1213.97)

## 2. ARPU: Average Revenue Per User

### 2.1 Total Revenue

In [21]:
total_revenue = payments_202303['receipt_amount'].sum()
total_revenue

np.int64(83033)

### 2.2 Number of Customers

In [22]:
number_of_customers = payments_202303['customer_id'].nunique()
number_of_customers

292

### 2.3 ARPU Calculation

In [23]:
arpu = round(total_revenue / number_of_customers, 2)
arpu

np.float64(284.36)

## 3. COGS: Cost of Goods Sold

### 3.1 Software expenses

In [24]:
production_expense = ['AWS Hosting', 'Google Cloud Storage', 'Atlassian Jira']
software_expense = expense_202303[expense_202303['item'].isin(production_expense)]['amount'].sum()
software_expense

np.int64(14200)

In [25]:
shared_software = ['Slack', 'Zoom']
shared_expense = expense_202303[expense_202303['item'].isin(shared_software)]['amount'].sum() * 0.6
shared_expense

np.float64(864.0)

### 3.2 Production Salary

In [26]:
production_salary = payroll_202303[payroll_202303['department'] == 'Engineering']['paid'].sum()
production_salary

np.int64(5200)

### 3.3 COGS Calculation

In [27]:
cogs = software_expense + shared_expense + production_salary
cogs

np.float64(20264.0)

## 4. Gross Margin

In [28]:
gross_margin = round((total_revenue - cogs)/total_revenue * 100, 2)
gross_margin

np.float64(75.6)

## 5. LTV - Life Time Value

In [29]:
lifespan['lifespan_days'] = pd.to_datetime(lifespan['churn_date']) - pd.to_datetime(lifespan['start_date'])
lifespan['lifespan_days'] = lifespan['lifespan_days'].dt.days
lifespan.sample(5)

,Unnamed: 0,start_date,churn_date,lifespan_days
98,1098,2020-06-17,2022-06-13,726
49,1049,2020-03-27,2021-03-19,357
72,1072,2020-07-26,2021-05-19,297
24,1024,2023-01-11,2023-01-21,10
95,1095,2023-01-09,2023-02-28,50


In [30]:
avg_lifespan_month = lifespan['lifespan_days'].mean() / 30
avg_lifespan_month

np.float64(9.841333333333333)

In [31]:
ltv = round(arpu * avg_lifespan_month * gross_margin/100, 2)
ltv

np.float64(2115.65)

## 6. LTV / CAC

In [32]:
ltv_cac = round(ltv / cac, 2)
ltv_cac

np.float64(1.74)

## 7. Conclusion

- Customer Acquisition Cost (CAC):
The CAC represents how much it costs to acquire a single new customer. TechStream's CAC is $1,213.97 indicating the combined expenses from CRM tools, sales & marketing salaries, and daily campaign spend are moderately high. This is a crucial metric for measuring marketing efficiency.

- Average Revenue Per User (ARPU):
ARPU is $284.36, showing the average revenue generated from each active customer in March. This figure directly affects profitability and informs pricing strategies.

- Gross Margin:
At 75.6%, TechStream's gross margin indicates a healthy buffer between revenue and direct production costs (such as hosting, engineering salaries, and shared software). A margin above 60% is often considered strong for SaaS and software companies.

- Customer Lifetime Value (LTV):
The LTV is $2,115.65 meaning each customer is expected to generate this amount of value throughout their relationship with TechStream. This figure is critical in long-term planning.

- LTV to CAC Ratio (LTV/CAC):
TechStream's LTV/CAC ratio is 1.74, which means for every one dollar spent acquiring a customer, TechStream is getting $1.74 in return over that customer's lifetime. While TechStream is not losing money per customer, the current ratio is below the commonly recommended benchmark of 3.0, which signals strong and scalable unit economics.
=> This indicates TechStream's current customer acquisition strategy is not yet efficient enough to sustainably scale profitably.

## 8. Recommendations

Based on the TechStream's calculated Unit Economics for March 2023, here's a set of recommendations for the company:

a. Increase Customer Lifetime Value (LTV)
- Enhance Customer Retention by monitoring the churn signals and proactively engage at-risk users and offering the loyalty benefits, regular check-ins, or success support for long-term retention.
- Upsell and Expand Usage by introducing the premium plans, usage-based pricing, or bundles and educating users on underutilized features to drive more engagement and value.

b. Improve Gross Margin: Optimize Operational Costs such as:
- Review infrastructure (e.g., AWS, Google Cloud) and negotiate or right-size resources
- Assess software licenses (e.g., Jira, Slack, Zoom) to avoid redundant tools.

c. Reduce Customer Acquisition Cost (CAC)
- Identify and cut underperforming campaigns (low conversion, high cost).
- Reallocate budget to channels with lower CAC and higher ROI.